# Clasificador de imagenes de perros y gatos

## Integrantes del grupo
- Ramirez Mendoza, Jorge Luis
- Huaman Huamani, Alexander Fermin
- Obeso Sanchez, Aldo Alessandro

## Bibliotecas necesarias para el proyecto

In [ ]:
import tensorflow as tf #biblioteca principal de TensorFlow.
import tensorflow_datasets as tfds #conjuntos de datos predefinidos para el entrenamiento y evaluación de modelos
from tensorflow.keras.layers import Dense, Flatten #capas de la red neuronal
from tensorflow.keras.models import Sequential #modelo secuencial que combina capas en secuencia
from tensorflow.keras.applications import VGG16 #clase proporcionada por TensorFlow que implementa la arquitectura del modelo VGG16.

## Carga del conjunto de datos desde TensorFlow Datasets

In [ ]:
datos, metadatos = tfds.load('cats_vs_dogs', split='train[:80%]', shuffle_files=True, with_info=True, as_supervised=True)
# La función tfds.load devuelve dos valores: datos y metadatos. El valor datos contiene el conjunto de datos cargado, que consiste en las imágenes y las etiquetas. El valor metadatos contiene información adicional sobre el conjunto de datos, como el número total de imágenes, las etiquetas disponibles, entre otros detalles.
#'cats_vs_dogs' Es el nombre del conjunto de datos que se va a cargar.
# split='train[:80%]' Indica qué porción del conjunto de datos se utilizará para entrenamiento. En este caso, se especifica que se desea utilizar el 80% de las imágenes para entrenamiento. El 20% restante se reservará para la validación.
# shuffle_files=True Esta opción indica si se deben mezclar los archivos del conjunto de datos.
# with_info=True Esta opción indica que se desea obtener información adicional sobre el conjunto de datos, como metadatos, junto con los datos mismos
# as_supervised=True Esta opción especifica que se desea cargar los datos en forma de pares (imagen, etiqueta) donde la imagen es la entrada y la etiqueta es la clase (perro o gato) correspondiente

In [ ]:
datos_entrenamiento = datos.take(int(0.8 * metadatos.splits['train'].num_examples))
# datos: Es el conjunto de datos cargado, que contiene las imágenes y etiqueta
# metadatos.splits['train'].num_examples: Devuelve el número total de ejemplos en el subconjunto de entrenamiento
# int(0.8 * metadatos.splits['train'].num_examples): Calcula el 80% de los ejemplos de entrenamiento y redondea al entero más cercano
# datos.take( ): Toma los primeros n ejemplos del conjunto de datos, donde n es el número calculado anteriormente
# Esta línea de código selecciona el 80% de los ejemplos de entrenamiento del conjunto de datos completo y los asigna a la variable 
datos_validacion = datos.skip(int(0.8 * metadatos.splits['train'].num_examples))
# datos.skip( ): Omite los primeros n ejemplos del conjunto de datos, donde n es el número calculado anteriormente
# Esta línea de código omite el 80% de los ejemplos de entrenamiento del conjunto de datos completo y asigna el resto (20%) a la variable datos_validacion

## Normalizacion y redimension de las imagenes

In [ ]:
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

datos_entrenamiento = datos_entrenamiento.map(preprocess_image)
datos_validacion = datos_validacion.map(preprocess_image)

## Definicion del modelo base pre-entrenado utilizando la arquitectura VGG16

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
base_model.trainable = False
# Configuramos el modelo para que no entrene las capas del modelo base, estableciendo base_model.trainable = False

### Construccion del modelo completo agregando capas adicionales a la arquitectura base

In [ ]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

### Compilamos el modelo especificando el optimizador, la función de pérdida y las métricas a utilizar durante el entrenamiento

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Cargamos la extensión de TensorBoard para visualizar los registros de entrenamiento

In [ ]:
%load_ext tensorboard

### Definimos la ubicación para guardar los registros de TensorBoard

In [ ]:
log_dir = "logs/"

### Creamos un callback de TensorBoard para registrar los datos durante el entrenamiento

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

### Entrenamos el modelo utilizando los datos de entrenamiento y validación, especificando el número de épocas y el callback de TensorBoard

In [ ]:
model.fit(datos_entrenamiento.batch(32),
          validation_data=datos_validacion.batch(32),
          epochs=10,
          callbacks=[tensorboard_callback])
# Este código carga un conjunto de datos de perros y gatos, construye un modelo de clasificación de imágenes utilizando la arquitectura VGG16 como base pre-entrenada, y luego lo entrena utilizando el optimizador Adam

In [ ]:
%tensorboard --logdir logs/
# Aqui debia de iniciar el tensorBoard, pero el proyecto se hizo en GoogleColab y por algun motivo siempre salto el error 403 cada que se quizo caragar TensorBoard

### Se guarda el modelo entrenado obtenido

In [ ]:
model.save('modelo_perros_gatos.h5')